# <b> Assignment: </b> Application of <font color=red> RNNs (SimpleRNN, GRU, LSTM)</font> in predition of correct emojis for given sentences using a <font color=blue> functional API model </font>

In [5]:
import numpy as np
import pandas as pd

In [6]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2025-09-21 16:21:10--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-09-21 16:21:11--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-09-21 16:21:11--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [7]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [8]:
embedding_matrix={}
words_to_index={}
index_to_words={}
words=set()
with open("glove.6B.100d.txt",'r',encoding='utf-8') as f:

  for line in f:
    values=line.split()
    word=values[0]
    embedding=np.array(values[1:],dtype='float32')

    embedding_matrix[word]=embedding
    words.add(word)

i = 1
for w in sorted(words):
  words_to_index[w] = i
  index_to_words[i] = w
  i+=1

print(len(embedding_matrix))

400000


In [2]:
from sklearn.metrics.pairwise import cosine_similarity


def similarity(u,v):
  return np.squeeze(cosine_similarity(u.reshape(1,-1),v.reshape(1,-1)))

In [9]:
def sentences_to_indices(X,word_to_index,max_len):
  m=X.shape[0]
  X_indices=np.zeros((m,max_len))
  for i in range(m):
    words=X[i].lower().split()
    for j,w in enumerate(words):
      X_indices[i,j]=word_to_index[w]
  return X_indices

In [10]:
X=np.array(['funny','lets go party','food is ready for you'])
X_indices =sentences_to_indices(X,words_to_index,max_len=5)

In [11]:
!wget https://graphemy.ir/datasets/emojify.zip

--2025-09-21 16:29:47--  https://graphemy.ir/datasets/emojify.zip
Resolving graphemy.ir (graphemy.ir)... 5.9.122.2
Connecting to graphemy.ir (graphemy.ir)|5.9.122.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8171 (8.0K) [application/zip]
Saving to: ‘emojify.zip’

emojify.zip         100%[===================>]   7.98K  --.-KB/s    in 0s      

2025-09-21 16:29:49 (191 MB/s) - ‘emojify.zip’ saved [8171/8171]



In [12]:
!unzip emojify.zip

Archive:  emojify.zip
  inflating: emojify_data.csv        
  inflating: tes.csv                 
  inflating: tess.csv                
  inflating: tesss.csv               
  inflating: test_emoji.csv          
  inflating: testing.csv             
  inflating: train_emoji.csv         


In [13]:
df=pd.read_csv("train_emoji.csv",header=None)
df.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [14]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 23.6 MB/s eta 0:00:00


In [15]:
import emoji

emoji_dictionary = {"0": "\u2764\uFE0F",
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}

def label_to_emoji(label):

    return emoji.emojize(emoji_dictionary[str(label)],language='alias')

In [17]:
X_train=df[0].values
y_train=df[1].values

In [18]:
index=2

print(X_train[index],y_train[index],label_to_emoji(y_train[index]))

It is the worst day in my life 3 😞


In [19]:
test_df=pd.read_csv("test_emoji.csv",header=None)
test_df.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [20]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_layer(embedding_matrix,words_to_index):
  vocab_len=len(words_to_index) + 1 # 0:padding
  emb_dim=embedding_matrix['ali'].shape[0]

  emb_matrix=np.zeros((vocab_len,emb_dim))

  for word,index in words_to_index.items():
    emb_matrix[index,:]=embedding_matrix[word]

  embedding_layer=Embedding(vocab_len,emb_dim,trainable=False)
  embedding_layer.build((None,))
  embedding_layer.set_weights([emb_matrix])

  return embedding_layer

In [21]:
e=pretrained_embedding_layer(embedding_matrix,words_to_index)
e

<Embedding name=embedding, built=True>

In [22]:
e.weights

[<Variable path=embedding/embeddings, shape=(400001, 100), dtype=float32, value=[[ 0.         0.         0.        ...  0.         0.         0.       ]
  [ 0.38472    0.49351    0.49096   ...  0.026263   0.39052    0.52217  ]
  [ 0.22657    0.64651    0.84828   ...  0.54712    0.7697     0.35075  ]
  ...
  [ 0.14335    0.557     -0.68806   ...  0.10501   -0.49575    0.39039  ]
  [-0.036419  -0.63433   -0.26185   ...  0.25043    0.21037    0.75933  ]
  [ 0.32008    0.21479   -0.036466  ...  0.088318   0.11623    0.0020262]]>]

In [36]:
# Functional API
from tensorflow.keras.layers import Input,AveragePooling1D,Dense,Flatten
from tensorflow.keras.models import Model

In [24]:
max(X_train,key=len)

'I am so impressed by your dedication to this project'

In [25]:
len(max(X_train,key=len))

52

In [26]:
max_len=len(max(X_train,key=len).split())

In [27]:
X_train_sequences=sentences_to_indices(X_train,words_to_index,max_len=max_len)
X_train[1],X_train_sequences[1]

('I am proud of your achievements',
 array([185457.,  52943., 293982., 268046., 394565.,  45460.,      0.,
             0.,      0.,      0.]))

#<b> Without <font color=red>a RNN layer </font>

In [79]:
input=Input((max_len,),dtype=np.int32)
embedding_layer=pretrained_embedding_layer(embedding_matrix,words_to_index)


avg=AveragePooling1D(pool_size=10)

flt=Flatten()

dense=Dense(5,activation='softmax')

x=embedding_layer(input)
x=avg(x)
x=flt(x)
x=dense(x)

model=Model(input,x)

model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

model.fit(X_train_sequences,y_train,epochs=50,batch_size=32,shuffle=True)

x_test=np.array(['I am happy','not feeling happy', 'I want to die'])
x_test_sequences=sentences_to_indices(x_test,words_to_index,max_len)
pred=model.predict(x_test_sequences)
pred

for item,label in zip(x_test,np.argmax(pred,axis=1)):
  print(item,label,label_to_emoji(label))

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.2041 - loss: 1.6950 
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1815 - loss: 1.6802
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1837 - loss: 1.6662
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1845 - loss: 1.6547
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.1795 - loss: 1.6390
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1901 - loss: 1.6265
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2266 - loss: 1.6063
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2542 - loss: 1.6040
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2233 - loss: 1.5968
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2452 - loss: 1.5792
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2702 - loss: 1.5843
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2295 - loss: 1.5754


In [80]:
model.evaluate(X_train_sequences,y_train)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4475 - loss: 1.4851  


[1.4973281621932983, 0.4166666567325592]

#<b> With a <font color=red>Single LSTM layer </font> including 200 memory cells

In [76]:
from tensorflow.keras.layers import LSTM

input=Input((max_len,),dtype=np.int32)
embedding_layer=pretrained_embedding_layer(embedding_matrix,words_to_index)


avg=AveragePooling1D(pool_size=10)

# A LSTM layer
lstm = LSTM(200, return_sequences=False)

flt=Flatten()

dense=Dense(5,activation='softmax')

x=embedding_layer(input)
x=avg(x)
x=lstm(x)
x=flt(x)
x=dense(x)

model=Model(input,x)

model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

model.fit(X_train_sequences,y_train,epochs=50,batch_size=32,shuffle=True)

x_test=np.array(['I am happy','not feeling happy', 'I want to die'])
x_test_sequences=sentences_to_indices(x_test,words_to_index,max_len)
pred=model.predict(x_test_sequences)
pred

for item,label in zip(x_test,np.argmax(pred,axis=1)):
  print(item,label,label_to_emoji(label))

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accuracy: 0.1180 - loss: 1.6085
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1450 - loss: 1.6056 
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2223 - loss: 1.6029
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2841 - loss: 1.6003 
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.3265 - loss: 1.5998
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2850 - loss: 1.6007
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3160 - loss: 1.5997
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2761 - loss: 1.6003 
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3131 - loss: 1.5941
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3326 - loss: 1.5911
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3666 - loss: 1.5888
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3683 - loss: 1.5908

In [77]:
model.evaluate(X_train_sequences,y_train)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3037 - loss: 1.5430 


[1.5549503564834595, 0.3030303120613098]

#<b> With a<font color=red> Single GRU layer </font> including 200 memory cells

In [74]:
from tensorflow.keras.layers import GRU

input=Input((max_len,),dtype=np.int32)
embedding_layer=pretrained_embedding_layer(embedding_matrix,words_to_index)


avg=AveragePooling1D(pool_size=10)

# A GRU layer
gru = GRU(200, return_sequences=False)

flt=Flatten()

dense=Dense(5,activation='softmax')

x=embedding_layer(input)
x=avg(x)
x=gru(x)
x=flt(x)
x=dense(x)

model=Model(input,x)

model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

model.fit(X_train_sequences,y_train,epochs=50,batch_size=32,shuffle=True)

x_test=np.array(['I am happy','not feeling happy', 'I want to die'])
x_test_sequences=sentences_to_indices(x_test,words_to_index,max_len)
pred=model.predict(x_test_sequences)
pred

for item,label in zip(x_test,np.argmax(pred,axis=1)):
  print(item,label,label_to_emoji(label))



Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.0704 - loss: 1.6276
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0968 - loss: 1.6193
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1291 - loss: 1.6151
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1158 - loss: 1.6119
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2658 - loss: 1.6007
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3076 - loss: 1.5948 
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2781 - loss: 1.5986
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3146 - loss: 1.5891 
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3089 - loss: 1.5863 
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3123 - loss: 1.5851
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3482 - loss: 1.5767 
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3668 - loss: 1.5708
E

In [75]:
model.evaluate(X_train_sequences,y_train)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.4293 - loss: 1.5080 


[1.5241893529891968, 0.4166666567325592]

#<b> With a <font color=red>Single SimpleRNN layer </font> including 200 memory cells

In [71]:
from tensorflow.keras.layers import SimpleRNN

input=Input((max_len,),dtype=np.int32)
embedding_layer=pretrained_embedding_layer(embedding_matrix,words_to_index)


avg=AveragePooling1D(pool_size=10)

# A SimpleRNN layer
srnn = SimpleRNN(200, return_sequences=False)

flt=Flatten()

dense=Dense(5,activation='softmax')

x=embedding_layer(input)
x=avg(x)
x=srnn(x)
x=flt(x)
x=dense(x)

model=Model(input,x)

model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

model.fit(X_train_sequences,y_train,epochs=50,batch_size=32,shuffle=True)

x_test=np.array(['I am happy','not feeling happy', 'I want to die'])
x_test_sequences=sentences_to_indices(x_test,words_to_index,max_len)
pred=model.predict(x_test_sequences)
pred

for item,label in zip(x_test,np.argmax(pred,axis=1)):
  print(item,label,label_to_emoji(label))

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.2227 - loss: 1.6309
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2903 - loss: 1.5921 
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2784 - loss: 1.5710 
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3099 - loss: 1.5599 
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2685 - loss: 1.5516 
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3164 - loss: 1.5496 
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3009 - loss: 1.5472 
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3264 - loss: 1.5239 
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2981 - loss: 1.5201 
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3167 - loss: 1.5014 
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2906 - loss: 1.5371 
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3036 - loss: 1.5039 
Ep

In [73]:
model.evaluate(X_train_sequences,y_train)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4942 - loss: 1.3886 


# Summury:
without a RNN layer:
accuracy: 0.4475 - loss: 1.4851
I am happy 3 😞
not feeling happy 3 😞
I want to die 3 😞
------------------------------------------------
With a SimpleRNN layer:
accuracy: 0.4942 - loss: 1.3886
I am happy 2 😄
not feeling happy 2 😄
I want to die 3 😞
------------------------------------------------
With a GRU layer:
accuracy: 0.4293 - loss: 1.5080
I am happy 2 😄
not feeling happy 2 😄
I want to die 3 😞
------------------------------------------------
With a LSTM layer:
accuracy: 0.3037 - loss: 1.5430
I am happy 2 😄
not feeling happy 2 😄
I want to die 2 😄
------------------------------------------------

#<b><font color=red> Note:</font>
Considering that the best performance was obtained by SimpleRNN, a bilayer SimpleRNN in combination of a double dropout layer was deemed as follows:

In [81]:
from tensorflow.keras.layers import Dropout

input=Input((max_len,),dtype=np.int32)
embedding_layer=pretrained_embedding_layer(embedding_matrix,words_to_index)


avg=AveragePooling1D(pool_size=10)

# A SimpleRNN layer
srnn1 = SimpleRNN(200, return_sequences=True)
srnn2 = SimpleRNN(100, return_sequences=False)
dropout1= Dropout(0.3)
dropout2= Dropout(0.3)
flt=Flatten()

dense=Dense(5,activation='softmax')

x=embedding_layer(input)
x=avg(x)
x=srnn1(x)
x=dropout1(x)
x=srnn2(x)
x=dropout2(x)
x=flt(x)
x=dense(x)

model=Model(input,x)

model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

model.fit(X_train_sequences,y_train,epochs=50,batch_size=32,shuffle=True)

x_test=np.array(['I am happy','not feeling happy', 'I want to die'])
x_test_sequences=sentences_to_indices(x_test,words_to_index,max_len)
pred=model.predict(x_test_sequences)
pred

for item,label in zip(x_test,np.argmax(pred,axis=1)):
  print(item,label,label_to_emoji(label))

model.evaluate(X_train_sequences,y_train)

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.2057 - loss: 1.6874
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2089 - loss: 1.6331 
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1912 - loss: 1.6238 
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2453 - loss: 1.6056
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3032 - loss: 1.5344
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1687 - loss: 1.6108 
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2379 - loss: 1.5736 
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3353 - loss: 1.5496 
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3048 - loss: 1.5407 
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2625 - loss: 1.5713 
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2983 - loss: 1.5677
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2680 - loss: 1.5812
E

[1.3098517656326294, 0.5151515007019043]